In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

In [2]:
vgg16 = VGG16(weights='imagenet')
print(vgg16.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [7]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

# Définition du modèle VGG16
classifier_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# On définit les couches cachées comme non entraînables
#(on pourrait les réentrainer ou choisir de ne réentrainer que les premières par exemple)
for layer in classifier_vgg16.layers:
    layer.trainable = False

# Ajout de nouvelles couches spécifiques à notre problème de classification
flat = Flatten()(classifier_vgg16.output)
classif = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat)
output = Dense(1, activation='sigmoid')(classif)

classifier_vgg16 = Model(inputs=classifier_vgg16.inputs, outputs=output)

classifier_vgg16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('images/images_augmentees',
                                                 target_size = (150, 150),
                                                 batch_size = 6,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('images/test',
                                            target_size = (150, 150),
                                            batch_size = 6,
                                            class_mode = 'binary')

classifier_vgg16.fit(training_set,
                     steps_per_epoch = 1000, #6000/6=1000
                     epochs = 8,
                     validation_data = test_set,
                     validation_steps = 20) #120/6=20

Found 6000 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/8
1000/1000 [==============================] - 411s 410ms/step - loss: 0.2335 - accuracy: 0.9017 - val_loss: 0.2387 - val_accuracy: 0.9500
Epoch 2/8
1000/1000 [==============================] - 412s 412ms/step - loss: 0.1345 - accuracy: 0.9462 - val_loss: 0.2259 - val_accuracy: 0.9417
Epoch 3/8
1000/1000 [==============================] - 395s 395ms/step - loss: 0.1051 - accuracy: 0.9585 - val_loss: 0.3261 - val_accuracy: 0.9417
Epoch 4/8
1000/1000 [==============================] - 407s 407ms/step - loss: 0.1009 - accuracy: 0.9595 - val_loss: 0.3035 - val_accuracy: 0.9083
Epoch 5/8
1000/1000 [==============================] - 407s 407ms/step - loss: 0.0760 - accuracy: 0.9702 - val_loss: 0.3609 - val_accuracy: 0.9167
Epoch 6/8
1000/1000 [==============================] - 424s 424ms/step - loss: 0.0721 - accuracy: 0.9727 - val_loss: 0.3741 - val_accuracy: 0.9417
Epoch 7/8
1000/1000 [==============

In [17]:
import numpy as np
from keras.preprocessing import image

def new_prediction(path):
    test_image = image.load_img(path, target_size = (150, 150))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier_vgg16.predict(test_image)

    if result[0][0] == 1:
        return 'plastic_bottle'
    elif result[0][1] == 1:
        return 'plastic_cups'
    else:
        return 'plastic_cutlery'

In [18]:
new_prediction('images/clean_single_prediction/378003_JPG.jpg')


'plastic_bottle'

In [20]:
new_prediction('images/clean_single_prediction/1649426291218.jpg')

'plastic_bottle'

In [21]:
new_prediction('images/clean_single_prediction/verre-a-shot-en-plastique-20ml-paquet-de-50utes.jpg')

'plastic_cups'

In [22]:
new_prediction('images/clean_single_prediction/fourchette-coquille-scaled.jpg')

'plastic_bottle'

In [23]:
new_prediction('images/clean_single_prediction/fourch.jpg')

'plastic_cutlery'

In [24]:
new_prediction('images/clean_single_prediction/lot-de-20-couteaux-en-plastique-rose_ADTISACOUVERT1Rose_1.jpg')

'plastic_cutlery'

In [10]:
# PREDICTION EN BINAIRE CLEAN/DIRTY
import numpy as np
from keras.preprocessing import image

def new_prediction(path):
    test_image = image.load_img(path, target_size = (150, 150))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier_vgg16.predict(test_image)
    #training_set.class_indices

    if result[0][0] == 1:
        return 'dirty'
    else:
        return 'clean'

In [11]:
new_prediction('images/predictions/bouteille_propre.jpg')

'clean'

In [12]:
new_prediction('images/predictions/bouteille_sale.jpg')

'dirty'

In [13]:
new_prediction('images/predictions/bouteille2_propre.jpg')

'clean'

In [14]:
new_prediction('images/predictions/bouteille2_sale.jpg')

'clean'

In [15]:
new_prediction('images/predictions/bouteille3_propre.jpg')

'clean'

In [16]:
new_prediction('images/predictions/bouteille3_sale.jpg')

'clean'

In [17]:
new_prediction('images/predictions/bouteille4_propre.jpg')

'clean'

In [18]:
new_prediction('images/predictions/bouteille4_sale.jpg')

'dirty'

In [19]:
new_prediction('images/predictions/bouteille5_propre.jpg')

'clean'

In [20]:
new_prediction('images/predictions/bouteille5_sale.jpg')

'dirty'

In [21]:
new_prediction('images/predictions/couvert_propre2_validation.jpg')

'clean'

In [22]:
new_prediction('images/predictions/couvert_propre3_validation.jpg')

'clean'

In [23]:
new_prediction('images/predictions/couvert_propre4_validation.jpg')

'clean'

In [24]:
new_prediction('images/predictions/couvert_propre5_validation.jpg')

'clean'

In [25]:
new_prediction('images/predictions/couvert_propre6_validation.jpg')

'clean'

In [26]:
new_prediction('images/predictions/couvert_propre7_validation.jpg')

'clean'

In [27]:
new_prediction('images/predictions/couvert_sale1_validation.jpg')

'dirty'

In [28]:
new_prediction('images/predictions/couvert_sale2_validation.jpg')

'dirty'

In [29]:
new_prediction('images/predictions/couvert_sale3_validation.jpg')

'dirty'

In [30]:
new_prediction('images/predictions/couvert_sale4_validation.jpg')

'clean'

In [31]:
new_prediction('images/predictions/couvert_sale5_validation.jpg')

'dirty'

In [32]:
new_prediction('images/predictions/couvert_sale6_validation.jpg')

'clean'

In [33]:
new_prediction('images/predictions/verre1_sale.jpg')

'clean'

In [34]:
new_prediction('images/predictions/verre3_propre.jpg')

'clean'

In [35]:
new_prediction('images/predictions/verre4_sale.jpg')

'dirty'

In [36]:
new_prediction('images/predictions/verre6_sale.jpg')

'dirty'

In [37]:
new_prediction('images/predictions/verre7_sale.jpg')

'dirty'

In [38]:
new_prediction('images/predictions/verre7_propre.jpg')

'clean'

In [39]:
classifier_vgg16.save('my_model.h5')

In [ ]:
def clean_or_dirty(path):
    res = new_prediction(path)
    return res